In [1]:
#2/26 - going to try to re calculate St Johns so we can understand why it's all 0s
#issue was it was trying to regex match parentheses. now will search for parentheses and replace with escape characters and should work.
#2/26 - need to exclude invalid transitions from individual team stats too
#in v02, we might try to save individual team time distributions too

In [2]:
import numpy as np
import pandas as pd
from itertools import product
from helper_functions.clean_transitions import clean_transition_column
from tqdm import tqdm

In [3]:
def validate_transitions(transition_df):
    #delete all transitions from FREETHROW state that don't result in either team r or opposite team i
    #also delete when freethrow turns to i2 or i3
    validate = []
    for i in range(len(transition_df)):
        curr_tran = transition_df['Transition'][i]
        if curr_tran in [('Bi1', 'Af3'),
                        ('Ai2', 'Bf3'),
                        ('Af0', 'Af3'),
                        ('Ai0', 'Bf3'),
                        ('Bf2', 'Bf2'),
                        ('Bi3', 'Af3'),
                        ('Ai0', 'Bf2'),
                        ('Bf3', 'Bf0'),
                        ('Af3', 'Af1'),
                        ('Af3', 'Af0'),
                        ('Bf0', 'Bf2'),
                        ('Bf3', 'Bf1'),
                        ('Ai1', 'Bf3'),
                        ('Br0', 'Af3'),
                        ('Ar0', 'Bf3'),
                        ('Af0', 'Af2'),
                        ('Ai3', 'Bf2'),
                        ('Bi3', 'Af2'),
                        ('Bf2', 'Bf1'),
                        ('Bf2', 'Bf0'),
                        ('Ai2', 'Bf2'),
                        ('Ai1', 'Bf2'),
                        ('Af2', 'Af1'),
                        ('Bi0', 'Af2'),
                        ('Bi2', 'Af2'),
                        ('Af2', 'Af0'),
                        ('Bi1', 'Af2'),
                        ('Bi3', 'Bi3'),
                        ('Ai3', 'Ai3'),
                        ('Ai0', 'Ai3'),
                        ('Bi0', 'Bi3'),
                        ('Ai2', 'Ai3'),
                        ('Ai3', 'Ai2'),
                        ('Ai3', 'Bf0'),
                        ('Br0', 'Af2'),
                        ('Ar0', 'Bf2'),
                        ('Ai1', 'Ai3'),
                        ('Bi3', 'Af0'),
                        ('Bi0', 'Af0'),
                        ('Ai0', 'Ai2'),
                        ('Ai1', 'Bf0'),
                        ('Bi3', 'Bi2'),
                        ('Ai0', 'Bf0'),
                        ('Ai2', 'Ai2'),
                        ('Bi1', 'Bi3'),
                        ('Bi0', 'Bi2'),
                        ('Bi2', 'Bi3'),
                        ('Bi1', 'Af0'),
                        ('Br0', 'Bi3'),
                        ('Ai1', 'Ai2'),
                        ('Bi2', 'Bi2'),
                        ('Bi1', 'Bi2'),
                        ('Ai2', 'Bf0'),
                        ('Ar0', 'Ai3'),
                        ('Bi2', 'Af0'),
                        ('Br0', 'Bi2'),
                        ('Br0', 'Af0'),
                        ('Ar0', 'Ai2'),
                        ('Ar0', 'Bf0'),
                        ('Bi2','Af3'),
                        ('Af3','Af2')]:
            validate.append(False)

        elif ('f' in curr_tran[0])&('f' in curr_tran[1]):
            if (('A' in curr_tran[0])&('A' in curr_tran[1])) | (('B' in curr_tran[0])&('B' in curr_tran[1])):
                validate.append(True)
            else:
                validate.append(False)
        elif (('Af' in curr_tran[0])&('Ai' in curr_tran[1])) | (('Bf' in curr_tran[0])&('Bi' in curr_tran[1])):
            validate.append(False)
        elif ('f' in curr_tran[0])&(('i3' in curr_tran[1]) | ('i2' in curr_tran[1])):
            validate.append(False)
        elif ('1' in curr_tran[1])&('f' not in curr_tran[0]):
            validate.append(False)
        else:
            validate.append(True)
    return validate

In [4]:
possible_states = ['Ai0',
'Ai1',
'Ai2',
'Ai3',
'Ar0',
'Af0',
'Af1',
'Af2',
'Af3',
'Bi0',
'Bi1',
'Bi2',
'Bi3',
'Br0',
'Bf0',
'Bf1',
'Bf2',
'Bf3']

possible_transitions = list(product(possible_states,possible_states))

In [10]:
# data = pd.read_excel('ALL_VALID_TRANSITIONS_v03.xlsx')

In [6]:
data2=pd.read_csv('ALL_VALID_TRANSITIONS_v04.csv')

In [9]:
data=data2.copy()

In [11]:
team_data = pd.read_excel('Team_Names_Abbrs_v02.xlsx')

In [ ]:
transition_times = pd.read_excel('All_Transition_With_Times_Not_Aggregated_v04.xlsx')
# transition_times['Transition'] = clean_transition_column(transition_times['Transition'])
unique_valid_trans = data2['Transition'].unique()

In [ ]:
#some basic ground rules - any F to F transition should be 0 seconds
unique_trans = transition_times['Transition'].unique()
for t in unique_trans:
    if (('f' in t[0]) & ('f' in t[1]))|(t not in list(unique_valid_trans)):
        transition_times.loc[transition_times['Transition'] == t, 'Time'] = 0


In [12]:
teams_saved = []

In [16]:
#need to do this for each team twice - once as team A, once as team B

for m in tqdm(team_data['name']):
    n = m.replace('(', ('\(')).replace(')','\)')

    if n in teams_saved:
        raise Exception('Collision on {}!'.format(n))

    #one copy of smaller dataset for each A and B
    shrink_data_a = data[data['filename'].str.contains(n)].copy().reset_index(drop=True)
    shrink_data_b = shrink_data_a.copy()

    #ADDING LOGIC TO GET TEAM-SPECIFIC TIME DISTRIBUTION
    shrink_time_a = transition_times[transition_times['filename'].str.contains(n)].copy().reset_index(drop=True)
    shrink_time_b = shrink_time_a.copy()

    unique_games = shrink_data_a['filename'].unique()
    games_count = len(unique_games)
    #ignore this if - this was earlier for testing
    if games_count <= 100:
        for fname in unique_games:

            #team A
            if fname.find(n)>fname.find('vs'):
                shrink_data_a.loc[shrink_data_a['filename']==fname, 'Transition'] = shrink_data_a.loc[shrink_data_a['filename']==fname, 'Transition'].str.replace('A','TEMP')
                shrink_data_a.loc[shrink_data_a['filename']==fname, 'Transition'] = shrink_data_a.loc[shrink_data_a['filename']==fname, 'Transition'].str.replace('B','A')
                shrink_data_a.loc[shrink_data_a['filename']==fname, 'Transition'] = shrink_data_a.loc[shrink_data_a['filename']==fname, 'Transition'].str.replace('TEMP','B')
                #same logic for time df
                shrink_time_a.loc[shrink_time_a['filename']==fname, 'Transition'] = shrink_time_a.loc[shrink_time_a['filename']==fname, 'Transition'].str.replace('A','TEMP')
                shrink_time_a.loc[shrink_time_a['filename']==fname, 'Transition'] = shrink_time_a.loc[shrink_time_a['filename']==fname, 'Transition'].str.replace('B','A')
                shrink_time_a.loc[shrink_time_a['filename']==fname, 'Transition'] = shrink_time_a.loc[shrink_time_a['filename']==fname, 'Transition'].str.replace('TEMP','B')

            #team B
            if fname.find(n)<fname.find('vs'):
                shrink_data_b.loc[shrink_data_b['filename']==fname, 'Transition'] = shrink_data_b.loc[shrink_data_b['filename']==fname, 'Transition'].str.replace('A','TEMP')
                shrink_data_b.loc[shrink_data_b['filename']==fname, 'Transition'] = shrink_data_b.loc[shrink_data_b['filename']==fname, 'Transition'].str.replace('B','A')
                shrink_data_b.loc[shrink_data_b['filename']==fname, 'Transition'] = shrink_data_b.loc[shrink_data_b['filename']==fname, 'Transition'].str.replace('TEMP','B')
                #same logic for time df
                shrink_time_b.loc[shrink_time_b['filename']==fname, 'Transition'] = shrink_time_b.loc[shrink_time_b['filename']==fname, 'Transition'].str.replace('A','TEMP')
                shrink_time_b.loc[shrink_time_b['filename']==fname, 'Transition'] = shrink_time_b.loc[shrink_time_b['filename']==fname, 'Transition'].str.replace('B','A')
                shrink_time_b.loc[shrink_time_b['filename']==fname, 'Transition'] = shrink_time_b.loc[shrink_time_b['filename']==fname, 'Transition'].str.replace('TEMP','B')

        #now clean transition column
        shrink_data_a['Transition']=clean_transition_column(shrink_data_a['Transition'])
        shrink_data_b['Transition']=clean_transition_column(shrink_data_b['Transition'])

        #same for time df
        shrink_time_a['Transition']=clean_transition_column(shrink_time_a['Transition'])
        shrink_time_b['Transition']=clean_transition_column(shrink_time_b['Transition'])

        #need to validate transitions too
        shrink_data_a['valid_tran'] = validate_transitions(shrink_data_a)
        shrink_data_a=shrink_data_a[shrink_data_a['valid_tran']==True].copy().reset_index(drop=True)
        shrink_data_a=shrink_data_a.drop('valid_tran',axis=1)

        shrink_time_a['valid_tran'] = validate_transitions(shrink_time_a)
        shrink_time_a=shrink_time_a[shrink_time_a['valid_tran']==True].copy().reset_index(drop=True)
        shrink_time_a=shrink_time_a.drop('valid_tran',axis=1)

        shrink_data_b['valid_tran'] = validate_transitions(shrink_data_b)
        shrink_data_b=shrink_data_b[shrink_data_b['valid_tran']==True].copy().reset_index(drop=True)
        shrink_data_b=shrink_data_b.drop('valid_tran',axis=1)

        shrink_time_b['valid_tran'] = validate_transitions(shrink_time_b)
        shrink_time_b=shrink_time_b[shrink_time_b['valid_tran']==True].copy().reset_index(drop=True)
        shrink_time_b=shrink_time_b.drop('valid_tran',axis=1)


        #group and count transitions
        transitions_agg_a = shrink_data_a.groupby('Transition')['Period'].count().reset_index()
        transitions_agg_b = shrink_data_b.groupby('Transition')['Period'].count().reset_index()

        #add zero counts for the rest of the transitions for completeness
        for t in possible_transitions:
            if t not in list(transitions_agg_a['Transition'].values):
                curr_row = pd.DataFrame([[t, 0]], columns=['Transition','Period'])
                transitions_agg_a=pd.concat([transitions_agg_a,curr_row])

            if t not in list(transitions_agg_b['Transition'].values):
                curr_row = pd.DataFrame([[t, 0]], columns=['Transition','Period'])
                transitions_agg_b=pd.concat([transitions_agg_b,curr_row])

        #rename column to Count
        transitions_agg_a = transitions_agg_a.rename(columns={'Period':'Count'})
        transitions_agg_b = transitions_agg_b.rename(columns={'Period':'Count'})

        #starting state and ending state columns for easier pivot
        transitions_agg_a['Starting_State'] = [x[0] for x in transitions_agg_a['Transition']]
        transitions_agg_a['Ending_State'] = [x[1] for x in transitions_agg_a['Transition']]

        transitions_agg_b['Starting_State'] = [x[0] for x in transitions_agg_b['Transition']]
        transitions_agg_b['Ending_State'] = [x[1] for x in transitions_agg_b['Transition']]

        #take only necessary columns
        transitions_agg_a=transitions_agg_a[['Starting_State','Ending_State','Count']]
        transitions_agg_b=transitions_agg_b[['Starting_State','Ending_State','Count']]

        #pivot
        transition_matrix_a = transitions_agg_a.pivot(index='Starting_State', columns='Ending_State', values='Count').fillna(0)
        transition_matrix_b = transitions_agg_b.pivot(index='Starting_State', columns='Ending_State', values='Count').fillna(0)

        #divide along horizontal axis for total probability of 1
        transition_matrix_a = transition_matrix_a.div(transition_matrix_a.sum(axis=1), axis=0)
        transition_matrix_b = transition_matrix_b.div(transition_matrix_b.sum(axis=1), axis=0)

        #kill na's 
        transition_matrix_a=transition_matrix_a.fillna(0)
        transition_matrix_b=transition_matrix_b.fillna(0)

        transition_matrix_a.to_excel('team_specific_matrix/{}_A.xlsx'.format(m))
        transition_matrix_b.to_excel('team_specific_matrix/{}_B.xlsx'.format(m))

        #now process time df
        #find interquartile range of each transition's time durations
        quartiles_a = shrink_time_a.groupby('Transition')['Time'].quantile([0.1, 0.9]).unstack()
        quartiles_b = shrink_time_b.groupby('Transition')['Time'].quantile([0.1, 0.9]).unstack()

        #merge original DF with interquartile range df to filter
        df_merged_a = pd.merge(shrink_time_a, quartiles_a, left_on='Transition', right_index=True, suffixes=('', '_quartile'))
        df_merged_b = pd.merge(shrink_time_b, quartiles_b, left_on='Transition', right_index=True, suffixes=('', '_quartile'))

        filtered_df_a = df_merged_a[(df_merged_a['Time'] >= df_merged_a[0.1]) & (df_merged_a['Time'] <= df_merged_a[0.9])]
        filtered_df_b = df_merged_b[(df_merged_b['Time'] >= df_merged_b[0.1]) & (df_merged_b['Time'] <= df_merged_b[0.9])]
        
        smaller_ranged_times_a = filtered_df_a[['Transition','Time']].copy()
        smaller_ranged_times_b = filtered_df_b[['Transition','Time']].copy()

        new_df_a = pd.DataFrame()
        for t in tqdm(smaller_ranged_times_a['Transition'].unique()):
            temp_df = smaller_ranged_times_a[smaller_ranged_times_a['Transition']==t].groupby('Time').count().reset_index()
            temp_df['Freq'] = temp_df['Transition']/temp_df['Transition'].sum()
            temp_df['Transition']=str(t)
            temp_df=temp_df[['Transition','Time','Freq']].copy()
            new_df_a=pd.concat([new_df_a, temp_df]).reset_index(drop=True)

        new_df_b = pd.DataFrame()
        for t in tqdm(smaller_ranged_times_b['Transition'].unique()):
            temp_df = smaller_ranged_times_b[smaller_ranged_times_b['Transition']==t].groupby('Time').count().reset_index()
            temp_df['Freq'] = temp_df['Transition']/temp_df['Transition'].sum()
            temp_df['Transition']=str(t)
            temp_df=temp_df[['Transition','Time','Freq']].copy()
            new_df_b=pd.concat([new_df_b, temp_df]).reset_index(drop=True)

        new_df_a['Transition']=clean_transition_column(new_df_a['Transition'])
        new_df_b['Transition']=clean_transition_column(new_df_b['Transition'])

        new_df_a.to_excel('team_specific_times/{}_A.xlsx'.format(m))
        new_df_b.to_excel('team_specific_times/{}_B.xlsx'.format(m))

        teams_saved.append(m)

100%|██████████| 680/680 [04:27<00:00,  2.54it/s]
